In [2]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact
import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import datetime

## Conexão com o banco de dados

In [3]:
cnx = 'postgresql://postgres:postgres@db:5432/acessiview'
sqlalchemy.create_engine(cnx)

Engine(postgresql://postgres:***@db:5432/acessiview)

In [4]:
con = pg.connect(host='db', dbname= 'acessiview', user='postgres', password= 'postgres')

## Definição da Aplicação

Acessiview é uma aplicação simples, onde pessoas com deficiência podem encontrar estabelecimentos que tenham tags específicas que tratem algum tipo de acessibilidade dentro de suas lojas. Por exemplo, uma pessoa que necessita se locomovem com cadeira de rodas e quer ir em um estabelecimento específico, pode verificar se ele existe no sistema e, caso queira, ver se o estabelecimento tem uma tag para cadeirantes como "Rampas", ou no caso de um restaurante uma tag como "banheiros acessíveis para cadeirantes", essas tags são criadas no sistema, ou sugeridas para criação e as pessoas que forem nos estabelecimentos podem verificar a veracidade das tags que foram adicionadas, dando um like ou comentário como avaliação, logo, outras pessoas que forem usar o aplicativo para encontrar estabelecimentos podem checar a confiabilidade das tags de um estabelecimento pelas avaliações feitas ao mesmo.

## CRUDS


#### USUÁRIO - LÍNGUA - DEFICIÊNCIA

#### READ


In [7]:
flag=''

input_search = widgets.Text(
    value='',
    placeholder='Digite um nome',
    disabled=False
)

button_search = widgets.Button(description="Search", value='')

def on_button_clicked_query(b):
    query = ''
    input_text = input_search.value
    
    try:
        query = f"select usuario.cod, usuario_deficiencia.cod_deficiencia, usuario_lingua.cod_lingua from usuario inner join usuario_deficiencia on usuario.cod = usuario_deficiencia.cod_usuario inner join usuario_lingua on usuario.cod = usuario_lingua.cod_usuario"
        df = pd.read_sql_query(query, cnx)
        if(input_search.value == ''):   
            query = f"select usuario.nome, deficiencia.nome, lingua.nome from usuario, deficiencia, lingua, usuario_lingua_def as view1 where usuario.cod = view1.cod and deficiencia.cod = view1.cod_deficiencia and lingua.cod = view1.cod_lingua"
        else:
            query = f"select usuario.nome, deficiencia.nome, lingua.nome from usuario, deficiencia, lingua, usuario_lingua_def as view1 where usuario.cod = view1.cod and deficiencia.cod = view1.cod_deficiencia and lingua.cod = view1.cod_lingua and usuario.nome='{input_text}'"
            
        df = pd.read_sql_query(query, cnx)
        display(input_search, button_search, df)
        
    except:        
        display(input_search, button_search)    
        display("Não foi possível realizar a consulta.")
        
button_search.on_click(on_button_clicked_query)

output = widgets.Output()

with output:
    output.clear_output()
    display(input_search, button_search)

output

        
        

Output()

#### ACTIONS

In [20]:
#CAMPOS DO USUÁRIO
cod_usuario = widgets.IntText(
    value = 0,
    description='Cod Usuário: ',
    disabled=False
)

cod_def = widgets.IntText(
    value = 0,
    description='Cod Deficiencia: ',
    disabled=False
)

cod_lingua = widgets.IntText(
    value = 0,
    description='Cod Lingua: ',
    disabled=False
)


button_insert_lingua = widgets.Button(description="Inserir Língua", value='')
button_insert_deficiencia = widgets.Button(description="Inserir Def", value='')
button_delete_lingua = widgets.Button(description="Delete Língua", value='')
button_delete_deficiencia = widgets.Button(description="Delete Def", value='')

def on_button_clicked_insert_lingua(b):
    try:
        cursor = con.cursor()
        cursor.execute("insert into public.usuario_lingua(cod_lingua, cod_usuario) values (%s, %s)", (cod_lingua.value, cod_usuario.value))
                              
        cursor.query
        con.commit()
        df = pd.read_sql("select usuario.nome, deficiencia.nome, lingua.nome from usuario, deficiencia, lingua, usuario_lingua_def as view1 where usuario.cod = view1.cod and deficiencia.cod = view1.cod_deficiencia and lingua.cod = view1.cod_lingua", cnx)
        display(cod_usuario, cod_def, cod_lingua, df)
        display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)

    except:
        cursor.execute("ROLLBACK")   
        display(cod_usuario, cod_def, cod_lingua)
        display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)
        display("Não foi possível realizar a inserção. Verifique restrições.")
        
    finally:
        cursor.close()
        
def on_button_clicked_insert_deficiencia(b):
    try:
        cursor = con.cursor()
        cursor.execute("insert into public.usuario_deficiencia(cod_deficiencia, cod_usuario) values (%s, %s)", (cod_def.value, cod_usuario.value))
                              
        cursor.query
        con.commit()
        df = pd.read_sql("select usuario.nome, deficiencia.nome, lingua.nome from usuario, deficiencia, lingua, usuario_lingua_def as view1 where usuario.cod = view1.cod and deficiencia.cod = view1.cod_deficiencia and lingua.cod = view1.cod_lingua", cnx)
        display(cod_usuario, cod_def, cod_lingua, df)
        display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)

    except:
        cursor.execute("ROLLBACK")   
        display(cod_usuario, cod_def, cod_lingua)
        display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)
        display("Não foi possível realizar a inserção. Verifique restrições.")
        
    finally:
        cursor.close()
        
        

def on_button_clicked_delete_lingua(b):
    try:
        cursor= con.cursor()
        cursor.execute("delete from public.usuario_lingua where cod_lingua=%s and cod_usuario=%s", (cod_lingua.value, cod_usuario.value,))
       
        rows_deleted = cursor.rowcount
        con.commit()
        df = pd.read_sql("select usuario.nome, deficiencia.nome, lingua.nome from usuario, deficiencia, lingua, usuario_lingua_def as view1 where usuario.cod = view1.cod and deficiencia.cod = view1.cod_deficiencia and lingua.cod = view1.cod_lingua", cnx)
        display(cod_usuario, cod_def, cod_lingua, df)
        display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)
    except:
        cursor.execute("ROLLBACK")   
        display(cod_usuario, cod_def, cod_lingua)
        display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)
        display("Não foi possível realizar a deleção. Verifique restrições.")
            
    finally:
        cursor.close()
        
def on_button_clicked_delete_deficiencia(b):
    try:
        cursor= con.cursor()
        cursor.execute("delete from public.usuario_deficiencia where cod_deficiencia=%s and cod_usuario=%s", (cod_def.value, cod_usuario.value,))
       
        rows_deleted = cursor.rowcount
        con.commit()
        df = pd.read_sql("select usuario.nome, deficiencia.nome, lingua.nome from usuario, deficiencia, lingua, usuario_lingua_def as view1 where usuario.cod = view1.cod and deficiencia.cod = view1.cod_deficiencia and lingua.cod = view1.cod_lingua", cnx)
        display(cod_usuario, cod_def, cod_lingua, df)
        display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)
    except:
        cursor.execute("ROLLBACK")   
        display(cod_usuario, cod_def, cod_lingua)
        display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)
        display("Não foi possível realizar a deleção. Verifique restrições.")
            
    finally:
        cursor.close()
        
button_insert_lingua.on_click(on_button_clicked_insert_lingua)
button_insert_deficiencia.on_click(on_button_clicked_insert_deficiencia)
button_delete_lingua.on_click(on_button_clicked_delete_lingua)
button_delete_deficiencia.on_click(on_button_clicked_delete_deficiencia)

output = widgets.Output()

with output:
    output.clear_output()
    display(cod_usuario, cod_def, cod_lingua)
    display(button_insert_lingua, button_insert_deficiencia, button_delete_lingua, button_delete_deficiencia)
    


output

Output()